# PLN

- O desafio: como representar "palavras" com números
- A solução: **Bag-of-Words** = Transforme a palavra em um vetor, exemplo: Pokemon = [0,1,0,1,0,0,0]

Vamos transformar palavras em tokens numéricos!

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt

print(f"Tensorflow version: {tf.__version__}")

In [ ]:
VOCAB_SIZE = 1000

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=VOCAB_SIZE)

print(f"Temos {len(x_train)} treinamento/características e {len(x_test)} testes/alvos.")


print("\nExemplo de treinamento:\n")
print(x_train[0])

print("\nExemplo de alvo (Critica positiva):\n")
print(y_train[0])

## Decodificando Palavras (Demonstração)

In [ ]:
word_index = keras.datasets.imdb.get_word_index()

word_index = {k:(v+3) for k, v in word_index.items()}
word_index["[PAD]"] = 0
word_index["[START]"] = 1
word_index["[UNK]"] = 2
word_index["[UNUSED]"] = 3

reverse_word_index = {v: k for k, v in word_index.items()}

def decode_review(text_indexes):
  return ' '.join([reverse_word_index.get(i, '?') for i in text_indexes])

print(" --- Critica Decodificada --- ")
print(decode_review(x_train[0]))

In [ ]:
# Reduzindo críticas grandes demais (visando economizar processamento e filtrar comentário prolixos)
MAX_LEN=250

x_train_pad = pad_sequences(x_train, maxlen=MAX_LEN, padding='post')
x_test_pad = pad_sequences(x_test, maxlen=MAX_LEN, padding='post')

print(f'Tamanho de uma crítica grande (antes da compressão): {len(x_train[56])}') # no indice 56 temos uma crítica bem grande!
print(f'Conteúdo da crítica: {decode_review(x_train[56])}')

print(f'Tamanho da crítica após compressão: {len(x_train_pad[56])}') # 250
print(f'Conteúdo da crítica: {decode_review(x_train_pad[56])}')

In [ ]:
EMBBED_DIMENSION = 46

model = keras.Sequential([
  layers.Embedding(VOCAB_SIZE, EMBBED_DIMENSION, input_length=MAX_LEN),
  layers.Bidirectional(layers.LSTM(64)),
  layers.Dense(64, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(1, activation='sigmoid')
])

model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train_pad, y_train, epochs=5, batch_size=128, validation_data=(x_test_pad, y_test), verbose=1)

In [ ]:
test_loss, test_acc = model.evaluate(x_test_pad, y_test)
print(f'\nFalhas do test: {test_loss}, Acurácia do teste: {test_acc}')